In [1]:
#https://www.kaggle.com/thebrownviking20/bert-multiclass-classification/notebook#BERT-implementation
import numpy as np 
import pandas as pd
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys

from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
#from tensorflow.keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report

In [3]:
column_names = ['post', 'tags'] #data.csv

In [4]:
newdf = pd.read_csv('data.csv',
        header=0,
        na_values = '0',
        usecols=column_names,
        comment='\t',
        skipinitialspace=True)
newdf = newdf.replace(0, np.nan)
newdataset = newdf.copy()
newdataset.tail()

,post,tags
40024,BOTTLE MLDFLNTGLS 50MLX20MM GRE60-6037,ZPAK
40025,GLYCERIN USP,ZRAW
40026,TRAY LABEL REFACTO AF 500 UA,ZPAK
40027,VIAL LABEL BENEFIX 500 MA,ZPAK
40028,I-XALATAN EXS,ZPAK


In [5]:
one_hot = pd.get_dummies(newdataset["tags"])
newdataset.drop(['tags'],axis=1,inplace=True)
newdataset = pd.concat([newdataset,one_hot],axis=1)
newdataset.head()

,post,HALB,ZPAK,ZRAW
0,FS LYRICA 200MG 100 PCS RU,0,1,0
1,OPADRY II ORANGE 85F13934,0,0,1
2,CIRCULAR PHYSICIANS-2PGS FOLDED & GLUED,0,1,0
3,NORVASC TABS 10MG (PR)D BRAND LOGO,1,0,0
4,IS PROVERA CPR PE/BO,0,1,0


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(newdataset["post"].values, newdataset.drop(['post'],axis=1).values, test_size=0.2, random_state=42)

In [7]:
X_train.shape, y_train.shape

((32023,), (32023, 3))

In [8]:
X_test.shape, y_test.shape

((8006,), (8006, 3))

In [9]:
y_test[0]

array([0, 1, 0], dtype=uint8)

In [10]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=50)

In [11]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=50)

In [12]:
from tensorflow.keras.layers import Embedding
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))

W0917 20:38:56.156385 21736 deprecation.py:506] From C:\Users\gunawf01\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0917 20:38:56.332509 21736 deprecation.py:506] From C:\Users\gunawf01\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

W0917 20:39:00.258114 21736 deprecation.py:323] From C:\Users\gunawf01\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 28820 samples, validate on 3203 samples
Epoch 1/10
28820/28820 [==============================] - 57s 2ms/sample - loss: 1.0137 - acc: 0.5315 - val_loss: 0.7583 - val_acc: 0.7384
Epoch 2/10
28820/28820 [==============================] - 56s 2ms/sample - loss: 0.4171 - acc: 0.8598 - val_loss: 0.2573 - val_acc: 0.9129
Epoch 3/10
28820/28820 [==============================] - 55s 2ms/sample - loss: 0.1499 - acc: 0.9484 - val_loss: 0.1822 - val_acc: 0.9379
Epoch 4/10
28820/28820 [==============================] - 46s 2ms/sample - loss: 0.0870 - acc: 0.9722 - val_loss: 0.1715 - val_acc: 0.9426
Epoch 5/10
28820/28820 [==============================] - 42s 1ms/sample - loss: 0.0607 - acc: 0.9810 - val_loss: 0.1699 - val_acc: 0.9438
Epoch 6/10
28820/28820 [==============================] - 43s 1ms/sample - loss: 0.0460 - acc: 0.9860 - val_loss: 0.1720 - val_acc: 0.9460
Epoch 7/10
28820/28820 [==============================] - 40s 1ms/sample - loss: 0.0375 - acc: 0.9889 - val_loss: 0.1

In [15]:
score = model.evaluate(X_test, y_test,
                       batch_size=256, verbose=1)
print('Test accuracy:', score[1])

8006/8006 [==============================] - 3s 328us/sample - loss: 0.2162 - acc: 0.9419
Test accuracy: 0.94191855


In [16]:
preds = model.predict(X_test)
preds

array([[4.0111441e-02, 8.7411857e-01, 8.5770018e-02],
       [1.5025070e-03, 1.0446847e-03, 9.9745280e-01],
       [5.3265214e-04, 9.9930263e-01, 1.6480271e-04],
       ...,
       [9.9977714e-01, 3.1033556e-05, 1.9188956e-04],
       [3.1081207e-05, 9.0093936e-06, 9.9995995e-01],
       [3.7056481e-05, 6.9309433e-05, 9.9989367e-01]], dtype=float32)

In [17]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

             precision    recall  f1-score   support

          0       0.94      0.91      0.93      2658
          1       0.97      0.97      0.97      2637
          2       0.91      0.95      0.93      2711

avg / total       0.94      0.94      0.94      8006



In [18]:
np.argmax(preds[1])

2

In [19]:
X_test[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 3579])

In [44]:
sequences = tokenizer.texts_to_sequences(['FS XELJANZ 5MG 56 PCS PH'])
X_test_testing = pad_sequences(sequences, maxlen=50)
X_test_testing

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,  23, 505,  15, 326,  24,  75]])

In [45]:
preds1 = model.predict(X_test_testing)
preds1

array([[1.6350754e-06, 9.9999750e-01, 8.2952431e-07]], dtype=float32)

In [46]:
#HALB	ZPAK	ZRAW
np.argmax(preds1[0])

1